# データのグループ化、分割、ピボット

このノートでは、`DataFrame＃group_by` および `DataFrame＃pivot_table` 関数を使用してデータをグループ化、分割、ピボットする方法を説明します。


In [11]:
require 'daru'

false

まず、階層的にインデックス付けされたDataFrameを作成します。

In [12]:
# Create a multi-indexed DataFrame

tuples = [
  [:a,:one,:bar],
  [:a,:one,:baz],
  [:a,:two,:bar],
  [:a,:two,:baz],
  [:b,:one,:bar],
  [:b,:two,:bar],
  [:b,:two,:baz],
  [:b,:one,:foo],
  [:c,:one,:bar],
  [:c,:one,:baz],
  [:c,:two,:foo],
  [:c,:two,:bar]
]
multi_index = Daru::MultiIndex.from_tuples(tuples)

vector_arry1 = [11,12,13,14,11,12,13,14,11,12,13,14]
vector_arry2 = [1,2,3,4,1,2,3,4,1,2,3,4]

order_mi = Daru::MultiIndex.from_tuples([
  [:a,:one,:bar],
  [:a,:two,:baz],
  [:b,:two,:foo],
  [:b,:one,:foo]])

df_mi = Daru::DataFrame.new([
  vector_arry1, 
  vector_arry2, 
  vector_arry1, 
  vector_arry2], order: order_mi, index: multi_index)

#<Daru::DataFrame(12x4)>
                                  [:a, :one, [:a, :two, [:b, :two, [:b, :one,
          a        one        bar         11          1         11          1
                              baz         12          2         12          2
                   two        bar         13          3         13          3
                              baz         14          4         14          4
          b        one        bar         11          1         11          1
                   two        bar         12          2         12          2
                              baz         13          3         13          3
                   one        foo         14          4         14          4
          c        one        bar         11          1         11          1
                              baz         12          2         12          2
                   two        foo         13          3         13          3
                              bar         14          4         14          4

複数のインデックスを持つデータフレームから行を選択するには、完全なタプルを#row []メソッドに渡す必要があります。
部分タプルは、タプルと部分的に一致する行を部分的に返します。


In [13]:
# Specify complete tuple to choose a single row
df_mi.row[:a, :one,:bar]

#<Daru::Vector(4)>
               a
   a one bar  11
     two baz   1
   b two foo  11
     one foo   1

In [14]:
# Specify partial tuple to select index hierarchially
df_mi.row[:a]

#<Daru::DataFrame(4x4)>
                       [:a, :one, [:a, :two, [:b, :two, [:b, :one,
        one        bar         11          1         11          1
                   baz         12          2         12          2
        two        bar         13          3         13          3
                   baz         14          4         14          4

## グルーピングについて

`DataFrame#group_by` メソッドを使用すると、データフレーム内の要素を名前でグループ化できます。
これはSQL GROUP BYに似ています。


In [15]:
# See grouped rows with the 'groups' method

df = Daru::DataFrame.new({
  a: %w{foo bar foo bar   foo bar foo foo},
  b: %w{one one two three two two one three},
  c:   [1  ,2  ,3  ,1    ,3  ,6  ,3  ,8],
  d:   [11 ,22 ,33 ,44   ,55 ,66 ,77 ,88]
})

# Pass the vectors that are to be grouped in an Array to the group_by method. This 
# will return a Daru::Core::GroupBy object.
grouped = df.group_by([:a, :b])

# See the groups created using the 'groups' method.
grouped.groups

{["bar", "one"]=>[1], ["bar", "three"]=>[3], ["bar", "two"]=>[5], ["foo", "one"]=>[0, 6], ["foo", "three"]=>[7], ["foo", "two"]=>[2, 4]}

In [16]:
# First group by the columns :a and :b and then calculate mean of the grouped rows.
grouped.mean

#<Daru::DataFrame(6x2)>
                 c     d
   bar   one     2    22
       three     1    44
         two     6    66
   foo   one   2.0  44.0
       three     8    88
         two   3.0  44.0

`#get_group`メソッドは、特定のグループにアクセスするために使用できます。


In [17]:
grouped.get_group(["foo", "one"])

#<Daru::DataFrame(2x4)>
       a   b   c   d
   0 foo one   1  11
   6 foo one   3  77

## データのピボットについて

Excelのピボットテーブルと同様に、DataFrameは`#pivot_table`関数を提供し、特定の値を中心にデータをピボットし、洞察することができます。
いくつかの販売データを使って説明します。


In [18]:
sales = Daru::DataFrame.from_csv 'data/sales-funnel.csv'

#<Daru::DataFrame(17x8)>
               Account       Name        Rep    Manager    Product   Quantity      Price     Status
          0     714466 Trantow-Ba Craig Book Debra Henl        CPU          1      30000  presented
          1     714466 Trantow-Ba Craig Book Debra Henl   Software          1      10000  presented
          2     714466 Trantow-Ba Craig Book Debra Henl Maintenanc          2       5000    pending
          3     737550 Fritsch, R Craig Book Debra Henl        CPU          1      35000   declined
          4     146832 Kiehn-Spin Daniel Hil Debra Henl        CPU          2      65000        won
          5     218895  Kulas Inc Daniel Hil Debra Henl        CPU          2      40000    pending
          6     218895  Kulas Inc Daniel Hil Debra Henl   Software          1      10000  presented
          7     412290 Jerde-Hilp John Smith Debra Henl Maintenanc          2       5000    pending
          8     740150 Barton LLC John Smith Debra Henl        CPU          1      35000   declined
          9     141962 Herman LLC Cedric Mos Fred Ander        CPU          2      65000        won
         10     163416 Purdy-Kund Cedric Mos Fred Ander        CPU          1      30000  presented
         11     239344 Stokes LLC Cedric Mos Fred Ander Maintenanc          1       5000    pending
         12     239344 Stokes LLC Cedric Mos Fred Ander   Software          1      10000  presented
         13     307599 Kassulke,  Wendy Yule Fred Ander Maintenanc          3       7000        won
         14     688981 Keeling LL Wendy Yule Fred Ander        CPU          5     100000        won
        ...        ...        ...        ...        ...        ...        ...        ...        ...

`#pivot_table`メソッドは`index`というオプションを受け取ります。
`index`では、DataFrameのインデックスを作成するVectorを指定できます。


In [19]:
sales.pivot_table index: ['Manager', 'Rep']

#<Daru::DataFrame(5x3)>
                          Account   Quantity      Price
 Debra Henl Craig Book   720237.0       1.25    20000.0
            Daniel Hil   194874.0 1.66666666 38333.3333
            John Smith   576220.0        1.5    20000.0
 Fred Ander Cedric Mos   196016.5       1.25    27500.0
            Wendy Yule   614061.5        3.0    44250.0

また、 `：values`オプションを指定して、どのVectorを値として使用するかを指定することもできます。

`：vectors`オプションはピボットするカラムを指定します。

`：agg`オプションにはsum, mean, median, productといった統計メソッドを指定できます。


In [20]:
sales.pivot_table(index: ['Manager','Rep'], values: 'Price', vectors: ['Product'], agg: :sum)

#<Daru::DataFrame(5x4)>
                       ["Price",  ["Price",  ["Price",  ["Price", 
 Debra Henl Craig Book      65000      10000       5000        nil
            Daniel Hil     105000      10000        nil        nil
            John Smith      35000        nil       5000        nil
 Fred Ander Cedric Mos      95000      10000       5000        nil
            Wendy Yule     165000        nil       7000       5000